In [1]:
import math
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy.ma as ma
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import cartopy.crs as ccrs
import cartopy.crs as crs
import cartopy.feature as cfeature
import h5py  
from astropy.time import Time
import os
import glob
from osgeo import gdal
import scipy.ndimage
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC
import pprint
import csv

In [3]:
data = pd.read_csv("D:\\masters_data\\output\\csv\\Analysis\\New Folder\\All_CAR_data.csv", header=None)

data.columns = ["OID", "latitude","longitude","H", "V", "row","col", "mean_canopy", "min_canopy", 
                "max_canopy", "median_canopy","canopy_openness", "VI", "Yearly_rainfall", "LC",
                "First_burns", "Second_burns", "Total_burns", "Tile", "BA_category", 
                "canopy_spread","Rainfall_band200", "Rainfall_band500"]

#data['VI'] = int(data['VI']) / 10000

In [5]:
data = data.drop([data.index[0]])
data['latitude'] = data['latitude'].astype(np.float)
data['longitude'] = data['longitude'].astype(np.float)
data['mean_canopy'] = data['mean_canopy'].astype(np.float)
data['min_canopy'] = data['min_canopy'].astype(np.float)
data['max_canopy'] = data['max_canopy'].astype(np.float)
data['median_canopy'] = data['median_canopy'].astype(np.float)
data['canopy_openness'] = data['canopy_openness'].astype(np.float)
data['VI'] = data['VI'].astype(np.float)
data['LC'] = data['LC'].astype(np.int)
data['First_burns'] = data['First_burns'].astype(np.int)
data['Tile'] = data['Tile'].astype(str)
data['BA_category'] = data['BA_category'].astype(str)
data['canopy_spread'] = data['canopy_spread'].astype(np.float)
data['Yearly_rainfall'] = data['Yearly_rainfall'].astype(np.float)
data['Rainfall_band200'] = data['Rainfall_band200'].astype(str)
data['Rainfall_band500'] = data['Rainfall_band500'].astype(str)
data['H'] = data['H'].astype(str)
data['V'] = data['V'].astype(str)
data['row'] = data['row'].astype(str)
data['col'] = data['col'].astype(str)
data['VI'] = data['VI'] / 10000

In [6]:
data

,OID,latitude,longitude,H,V,row,col,mean_canopy,min_canopy,max_canopy,...,Yearly_rainfall,LC,First_burns,Second_burns,Total_burns,Tile,BA_category,canopy_spread,Rainfall_band200,Rainfall_band500
1,5,3.786884,17.957226,19,8,1490,1899,25.025993,6.172729,45.157928,...,1566.129034,2,0,0,0,H19V08,lowlow,1.557788,1500,above1500
2,6,3.785080,17.957041,19,8,1491,1899,26.307447,1.250824,41.558380,...,1566.129034,2,0,0,0,H19V08,lowlow,1.532173,1500,above1500
3,7,3.784178,17.956951,19,8,1491,1899,22.107281,1.025940,35.983459,...,1566.129034,2,0,0,0,H19V08,lowlow,1.581267,1500,above1500
4,8,3.650842,17.943571,19,8,1523,1897,23.171019,0.884308,38.741882,...,1585.140634,2,0,0,0,H19V08,lowlow,1.633833,1500,above1500
5,9,3.648136,17.943302,19,8,1523,1897,30.031158,2.544617,47.277313,...,1585.140634,2,0,0,0,H19V08,lowlow,1.489543,1500,above1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481620,1205668,5.221703,26.075653,20,8,1146,1431,6.738885,0.871399,17.202576,...,1536.017786,9,0,0,0,H20V08,lowlow,2.423424,1500,above1500
481621,1205669,5.220801,26.075562,20,8,1146,1431,4.576035,0.527954,16.362366,...,1536.017786,9,0,0,0,H20V08,lowlow,3.460291,1500,above1500
481622,1205670,5.217193,26.075195,20,8,1147,1431,4.869889,0.504517,12.951599,...,1536.017786,9,1,2,3,H20V08,medmed,2.555927,1500,above1500
481623,1205671,5.216290,26.075104,20,8,1147,1431,4.489307,0.531738,8.670471,...,1536.017786,9,1,2,3,H20V08,medmed,1.812915,1500,above1500


In [7]:
np.savetxt("D:\\masters_data\\output\\csv\\Analysis\\CAR_df.csv", data, delimiter=",", fmt='%s')

In [14]:
savannah = data[(data['LC'] == 8) | (data['LC'] == 9) ]

In [15]:
savannah

,OID,latitude,longitude,H,V,row,col,mean_canopy,min_canopy,max_canopy,...,Yearly_rainfall,LC,First_burns,Second_burns,Total_burns,Tile,BA_category,canopy_spread,Rainfall_band200,Rainfall_band500
7,1912,5.338834,19.840212,19,8,1118,2340,6.982491,1.247681,14.698914,...,1465.584180,9,1,4,5,H19V08,medmed,1.926423,1500,500_1500
8,1913,5.404875,19.833574,19,8,1102,2338,2.933857,0.573242,8.252380,...,1465.584180,9,1,6,7,H19V08,medhigh,2.617421,1500,500_1500
9,1914,5.405597,19.833506,19,8,1102,2338,2.819726,0.512360,7.634979,...,1465.584180,9,1,6,7,H19V08,medhigh,2.525997,1500,500_1500
10,1915,5.406500,19.833418,19,8,1101,2338,2.423932,0.503265,7.126129,...,1465.584180,9,1,7,8,H19V08,medhigh,2.732281,1500,500_1500
11,1916,5.407402,19.833324,19,8,1101,2338,3.226515,0.600067,9.371765,...,1465.584180,9,1,7,8,H19V08,medhigh,2.718629,1500,500_1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481620,1205668,5.221703,26.075653,20,8,1146,1431,6.738885,0.871399,17.202576,...,1536.017786,9,0,0,0,H20V08,lowlow,2.423424,1500,above1500
481621,1205669,5.220801,26.075562,20,8,1146,1431,4.576035,0.527954,16.362366,...,1536.017786,9,0,0,0,H20V08,lowlow,3.460291,1500,above1500
481622,1205670,5.217193,26.075195,20,8,1147,1431,4.869889,0.504517,12.951599,...,1536.017786,9,1,2,3,H20V08,medmed,2.555927,1500,above1500
481623,1205671,5.216290,26.075104,20,8,1147,1431,4.489307,0.531738,8.670471,...,1536.017786,9,1,2,3,H20V08,medmed,1.812915,1500,above1500


In [16]:
np.savetxt("D:\\masters_data\\output\\csv\\Analysis\\Savannah_df.csv", savannah, delimiter=",", fmt='%s')